### 1. Výpočet tepové frekvence z EKG signálu

In [1]:
import wfdb
from wfdb import Record
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import sys
import subprocess


def install_and_import(package):
    if package not in sys.modules:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    globals()[package] = __import__(package)


install_and_import("ipywidgets")
from ipywidgets import interact, FloatSlider

In [ ]:
# Vstupní data: https://physionet.org/content/butqdb/1.0.0/
# Testovací databáze: https://physionet.org/content/nsrdb/1.0.0/

In [47]:
def load_ecg(file_name: str) -> Record:
    ecg = wfdb.rdrecord(file_name)
    return ecg


def ecg_info(ecg: Record):
    info = [
        ["Název souboru", ecg.record_name],
        ["Vzorkovací frekvence (Hz)", ecg.fs],
        ["Délka signálu (vzorky)", ecg.sig_len],
        ["Počet kanálů", ecg.n_sig],
        ["Názvy kanálů", ", ".join(ecg.sig_name)],
        ["Jednotky pro každý kanál", ", ".join(ecg.units)],
    ]

    info_df = pd.DataFrame(info, columns=["Informace", "Hodnota"])
    display(info_df)


def ecg_plot(ecg: Record):
    signal = ecg.p_signal.ravel()
    total_length_sec = len(signal) / ecg.fs

    def plot_segment(start_sec, end_sec):
        if end_sec <= start_sec:
            print("Čas konce musí být větší než čas začátku.")
            return
        sample_start = int(start_sec * ecg.fs)
        sample_end = int(min(end_sec * ecg.fs, len(signal)))
        segment = signal[sample_start:sample_end]
        time_axis = np.arange(sample_start, sample_end) / ecg.fs
        plt.figure(figsize=(8, 5))
        plt.plot(time_axis, segment)
        plt.title(f"ECG: {start_sec:.1f} s až {end_sec:.1f} s")
        plt.xlabel("Čas (s)")
        plt.ylabel("Amplituda (µV)")
        plt.grid(True)
        plt.show()

    interact(
        plot_segment,
        start_sec=FloatSlider(
            min=0,
            max=max(0, total_length_sec - 1),
            step=1.0,
            value=0,
            description="Začátek (s)",
        ),
        end_sec=FloatSlider(
            min=1,
            max=total_length_sec,
            step=1.0,
            value=10.0,
            description="Konec (s)",
        ),
    )

In [48]:
ecg_100001 = load_ecg("100001_ECG")

In [49]:
ecg_info(ecg_100001)

,Informace,Hodnota
0,Název souboru,100001_ECG
1,Vzorkovací frekvence (Hz),1000
2,Délka signálu (vzorky),87087000
3,Počet kanálů,1
4,Názvy kanálů,ECG
5,Jednotky pro každý kanál,uV


In [50]:
# Při manuálním zadáváním hodnot nutné stisknout ENTER pro potvrzení.
# Při většinách hodnotách je nutné chvíli počkat.
ecg_plot(ecg_100001)

interactive(children=(FloatSlider(value=0.0, description='Začátek (s)', max=87086.0, step=1.0), FloatSlider(va…